In [18]:
import polars as pl 
import polars.selectors as cs
import seaborn as sns
import utils
import get_data
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.manifold import TSNE
import itertools
import umap 
from sklearn.cluster import KMeans
from sklearn.metrics import rand_score, adjusted_mutual_info_score, mutual_info_score, adjusted_rand_score

In [ ]:
reducer = umap.UMAP()

In [5]:
frame = pl.read_parquet(r"C:\Users\faune\Desktop\thesis\stanford_sentiment.parquet")

In [13]:
kmeans = KMeans(n_clusters=2)
frame = frame.with_columns(
    pl.Series(
        name= 'clusters',
        values= kmeans.fit_predict(frame.select('embeddings').to_series().to_list())
    )
)

In [22]:
print( 'rand score', rand_score(labels_true=frame.select('label').to_series().to_list(), labels_pred=frame.select('clusters').to_series().to_list()))
print( 'adjusted rand score', adjusted_rand_score(labels_true=frame.select('label').to_series().to_list(), labels_pred=frame.select('clusters').to_series().to_list()))
print('adjusted mutual info score', adjusted_mutual_info_score(labels_true=frame.select('label').to_series().to_list(), labels_pred=frame.select('clusters').to_series().to_list()))
print('mutual info score', mutual_info_score(labels_true=frame.select('label').to_series().to_list(), labels_pred=frame.select('clusters').to_series().to_list()))

rand score 0.8613028432987012
adjusted rand score 0.7225388486397475
adjusted mutual info score 0.6131038139937539
mutual info score 0.42014517791294226
